In [38]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [39]:
req = requests.get('http://www.erobertparker.com/newsearch/vintagechart1.aspx/VintageChart.aspx')

In [41]:
soup = BeautifulSoup(req.text, 'lxml')

In [42]:
charts = soup.find_all(attrs={'class':'chart'})

Get the labels as a dataframe.

In [142]:
labels = charts[0].find_all('tr')[1:-1]

label_list = [[(y.find('img'), y.get('rowspan')) 
               for y in x.find_all('td')] 
              for x in labels]

label_len = len(label_list)
empty_list = [np.NaN]*label_len
label_df = pd.DataFrame({'loc1':empty_list, 'loc2':empty_list, 'loc3':empty_list})

for col in range(3):
    pos = 0
    while pos < label_len:

        label = label_list[pos].pop(0)
        
        try:
            text = label[0]
            if text is None:
                text = np.NaN
            else:
                text = text.get('alt')
            
            nrows = label[1]
            if nrows is None:
                nrows = 1
            else:
                nrows = int(nrows)
            
            label_df.loc[pos:pos+nrows, 'loc'+str(col+1)] = text
            pos += nrows
        except:
            pos += 1

In [164]:
label_df.head()

,loc1,loc2,loc3
0,Europe,NaN,"Austria Riesling, Gruner Veltliner"
1,Europe,France,Alsace
2,Europe,France,Bordeaux: St. Julien/Pauillac St. Estephe
3,Europe,France,Margaux
4,Europe,France,Graves


In [165]:
label_df.shape

(51, 3)

Get the year labels for the chart.

In [156]:
year_list = [x.text.strip() for x in charts[1].find('tr').find_all('th')]

In [162]:
len(year_list)

41

Get the rankings for the chart and clean up the values.

In [206]:
ranking_df = pd.DataFrame([[y.text.strip() 
               for y in x.find_all('td')] 
              for x in charts[1].find_all('tr')[1:-1]]).loc[:,1:]

ranking_df.columns = year_list

ranking_df = pd.concat([label_df, ranking_df], axis=1).set_index(['loc1', 'loc2', 'loc3'])

ranking_df = ranking_df.replace('NT',np.NaN).replace('NV',np.NaN)

for col in ranking_df.columns:
    ranking_df[col] = ranking_df[col].str.replace(r"""[A-Z]+""", '')
    ranking_df[col] = ranking_df[col].apply(lambda x: float(x))

In [207]:
ranking_df

2014  \
loc1             loc2       loc3                                              
Europe           NaN        Austria Riesling, Gruner Veltliner          NaN   
                 France     Alsace                                      NaN   
                            Bordeaux: St. Julien/Pauillac St. Estephe   NaN   
                            Margaux                                     NaN   
                            Graves                                      NaN   
                            Pomerol                                     NaN   
                            Emilion                                     NaN   
                            Barsac/Sauternes                            NaN   
                            Côte de Nuits (Red)                         NaN   
                            Côte de Beaune (Red)                        NaN   
                            White Burgundy                              NaN   
                            Beaujolais                                  NaN   
                            Champagne                                   NaN   
                            Languedoc                                   NaN   
                            Roussillon                                  NaN   
                            Loire Valley (white)                        NaN   
                            North-Côte Rôtie/Hermitage                  NaN   
                            South-Châteauneuf du Pape                   NaN   
                 NaN        Germany: Mosel-Saar-Ruwer                   NaN   
                            Germany: Rhine (Riesling)                   NaN   
                 Italy      Campania Taurasi                            NaN   
                            Friuli: Collio                             85.0   
                            Piedmont: Barbaresco                        NaN   
                            Piedmont: Barbaresco                        NaN   
                            Trentino-Alto Adige                        87.0   
                            Tuscany: Bolgheri                           NaN   
                            Tuscany: Brunello di Montalcino             NaN   
                            Tuscany: Chianti Classico                   NaN   
                            Sicily: Etna                               97.0   
                            Veneto: Valpolicella (Amarone)              NaN   
                 NaN        Portugal                                    NaN   
                            Vintage Port                                NaN   
                 Spain      Rioja                                       NaN   
                            Ribera del Duero                            NaN   
                            Galicia                                     NaN   
USA              California Cabernet Sauvignon                         91.0   
                            Chardonnay                                 90.0   
                            Zinfandel                                  92.0   
                            Pinot Noir                                 94.0   
                            California Central Coast Rhône Rangers      NaN   
                            California Central Coast Rhône Rangers      NaN   
                 NaN        Oregon: Willamette Valley Pinot Noir        NaN   
                            Washington Cabernet Sauvignon and Syrah     NaN   
South Hemisphere NaN        Argentina                                   NaN   
                 Australia  S.Australia:Barossa/Clare/McLaren Vale      NaN   
                            Western Australia                           NaN   
                            New South Wales                             NaN   
                            Victoria / Tasmania                         NaN   
                 NaN        Chile                                       NaN   
                            New Zealand   

In [208]:
ranking_df.to_pickle('../pkl/05_vintage_years.pkl')